#《PyTorch深度学习实践》完结合集

`
https://www.bilibili.com/video/BV1Y7411d7Ys?p=9&vd_source=1e395a38862b4931fcb8904d5422cf65
`

# TensorBoard

In [65]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np

writer = SummaryWriter("logs")
for i in np.linspace(0.0, 1.0, 100):
    writer.add_scalar("y=log(x)3",  np.log(i), i)
writer.close()


C:\Users\Administrator\AppData\Local\Temp\ipykernel_10276\1296840023.py:6: RuntimeWarning: divide by zero encountered in log
  writer.add_scalar("y=log(x)3",  np.log(i), i)


# 反向传播

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

x_data = [i + 1.0 for i in range(3)]
y_data = [ 2*(i+1.0) for i in range(3)]

w = torch.Tensor([1.0])
w.requires_grad = True

def forward(x):
    return x * w
def loss(x, y):
    y_pred = forward(x)
    return (y_pred - y) ** 2


for epoch in range (100):
    for x, y in zip(x_data, y_data):
        l = loss(x, y)
        l.backward()
        print (l, x, y, w.grad)
        w.data = w.data - 0.01 * w.grad.item()
        w.grad.zero_()

forward(23)

# 基本模型
以及tensor变量
```
import torch

tensor = torch.reshape(torch.arange(1, 4, 1, dtype=torch.float32) * 2, (-1, 1))
# tensor = torch.Tensor([[1.0]])

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")
```

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt


x_data = torch.reshape(torch.arange(1, 4, 1,dtype=torch.float32), (-1, 1))
y_data = torch.reshape(torch.arange(1, 4, 1,dtype=torch.float32) * 2, (-1, 1))
# x_data = torch.Tensor([[1.0],[2.0],[3.0]])
# y_data = torch.Tensor([[1.0],[4.0],[6.0]])

class LinearModel(torch.nn.Module):
    def __init__(self) -> None:
        super(LinearModel, self).__init__()
        self.linear = torch.nn.Linear(1,1)
    def forward(self, x):
        return self.linear(x)

model =  LinearModel()
print(model)
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

for epoch in range (100):
    y_pred = model(x_data)
    loss = criterion(y_pred, y_data)
    print(epoch, loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

model(torch.Tensor([[2.0]])) # 乘2运算

# 分类函数
$loss = -(ylog \hat y + (1-y)log(1 - \hat y))$

当$\hat y$ 是1时， loss函数即为$ -(1-y)log(0.0) $

当$\hat y$ 是0时， loss函数即为$ -(y) $

In [ ]:
import torch
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt


x_data = torch.reshape(torch.arange(1, 4, 1,dtype=torch.float32), (-1, 1)) # 第1维就是批随意大小
# y_data = torch.reshape(torch.arange(1, 4, 1,dtype=torch.float32) * 2, (-1, 1))
# x_data = torch.Tensor([[1.0],[2.0],[3.0]])
y_data = torch.Tensor([[0.0],[0.0],[1.0]])

class LinearModel(torch.nn.Module):
    def __init__(self) -> None:
        super(LinearModel, self).__init__()
        self.linear = torch.nn.Linear(1,3) # 直接就是批运算， 第一维度指定输入数据的元素个数与形状
        self.linear2 = torch.nn.Linear(3,1)
    def forward(self, x):
        return F.sigmoid( self.linear2(self.linear(x)) )

model =  LinearModel()
print(model)
criterion = torch.nn.BCELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for epoch in range (10000):
    y_pred = model(x_data)  # 直接就是批运算
    loss = criterion(y_pred, y_data)
    print(epoch, loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
model(torch.Tensor([[1.0],[2.0],[3.0]])) # 作分类

# 多维数据处理

In [83]:
import pandas as pd
#  任意多维数组
a = [1,2,3]
b = [2,4,2]
c = [3,6,5]
# 字典中的key值即为CSV中的列表
dataframe = pd.DataFrame({'a_name':a, 'b_name':b, 'c_name':c})
# 是否要补充行序号,是否需要压缩
# dataframe.to_csv("test.csv.gz", index=False, sep=',',compression='gzip')
dataframe.to_csv("test.csv", index=False, sep=',')

In [ ]:
import numpy as np
import torch
xy = np.loadtxt('test.csv', delimiter=',', dtype=np.float32, skiprows=1)
x_data = torch.from_numpy(xy[:,:-1])
y_data = torch.from_numpy(xy[:, [-1]])
print(xy, x_data, y_data) # 3个样本，3个输出

class LinearModel(torch.nn.Module):
    def __init__(self) -> None:
        super(LinearModel, self).__init__()
        self.linear = torch.nn.Linear(2,3) # 输入两个元素，输出3个元素
        self.linear2 = torch.nn.Linear(3,1) # 输入3个元素，输出1个元素
    def forward(self, x):
        return self.linear2(self.linear(x))

model =  LinearModel()
print(model)
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for epoch in range (10000):
    y_pred = model(x_data)  # 直接就是批运算
    loss = criterion(y_pred, y_data)
    print(epoch, loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
model(torch.Tensor([[1.0, 4.0],[2.0,4.0],[3.0,4.0]])) # 做加法

# 实现数据集epoch与mini-batch训练

In [ ]:
import pandas as pd
#  任意多维数组
a = [np.random.randint(1, 13) for i in range(100)]
b = [np.random.randint(1, 13) for i in range(100)]
c = np.add(a,b)
# 字典中的key值即为CSV中的列表
dataframe = pd.DataFrame({'a_name':a, 'b_name':b, 'c_name':c})
# 是否要补充行序号,是否需要压缩
# dataframe.to_csv("test.csv.gz", index=False, sep=',',compression='gzip')
dataframe.to_csv("batch.csv", index=False, sep=',')


In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset,DataLoader
# 制作数据集
class MyData(Dataset):
    def __init__(self, root_dir, label_dir):
        xy = np.loadtxt('batch.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.x_data = torch.from_numpy(xy[:,:-1])
        self.y_data = torch.from_numpy(xy[:, [-1]])
        self.len = xy.shape[0]
        return
    def __getitem__(self, idx): # 取某一条数据
        return self.x_data[idx], self.y_data[idx]
    def __len__(self):
        return self.len

dataset = MyData("","")
# 指定一次取一批数据，同时shuffle并可以并发取
train_loader = DataLoader(dataset=dataset, batch_size=32, shuffle=True)

class LinearModel(torch.nn.Module):
    def __init__(self) -> None:
        super(LinearModel, self).__init__()
        self.linear = torch.nn.Linear(2,5) # 输入两个元素，输出3个元素
        self.linear2 = torch.nn.Linear(5,1) # 输入3个元素，输出1个元素
    def forward(self, x):
        return self.linear2(self.linear(x))

model =  LinearModel()
print(model)
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.00001)

for epoch in range (100):
    for i, data in enumerate(train_loader):
        # 1. prepare data
        inputs, lables = data
        # 2. forward
        y_pred = model(inputs)
        loss = criterion(y_pred, lables)
        print(epoch, i, loss.item())
        # 3.backward
        optimizer.zero_grad()
        loss.backward()
        # 4. update
        optimizer.step()

model(torch.Tensor([[1.0, 4.0],[2.0,4.0],[3.0,4.0]])) # 做加法


# 多分类问题
一次是输入多个样本的， 一次输出多分类值。使用交叉熵
交叉熵 $(P, Q) = - \sum P(i) log_2 Q(i) $

| animal | dog       | chicken   | bird      | fox       |
| ------ | --------- | --------- | --------- | --------- |
| label  | [1,0,0,0] | [0,1,0,0] | [0,0,1,0] | [0,0,0,1] |

对于同一张狗照片，机器预测结果ans1和ans2分别是[0.4, 0.3, 0.1, 0.1],[0.98, 0.01,0,0.01]

分别用交叉熵计算结果为$(P_1, Q_1) = - \sum P_1(i) log_2 Q_1(i) = -(1log0.4 + 0log0.3 + 0log0.1 + 0log0.1 = -log0.4) $

分别用交叉熵计算结果为$(P_1, Q_2) = - \sum P_1(i) log_2 Q_1(i) = -(1log0.98 + 0log0.01 + 0log0 + 0log0.01 \approx 0) $



In [121]:
import torch
criterion = torch.nn.CrossEntropyLoss()
labels = torch.LongTensor([2, 0, 1]) # 注意pytorch这里需要用LongTensor, 这里是选择器
y_pred1 = torch.Tensor([[0.1,0.2,0.9],  # 2
                        [1.1,0.1,0.2],  # 0
                        [0.2,2.1,0.1]]) # 1
y_pred2 = torch.Tensor([[0.8,0.2,0.3],  # 2
                        [0.2,0.3,0.5],  # 0
                        [0.2,0.2,0.5]]) # 1

l1 = criterion(y_pred1, labels)
l2 = criterion(y_pred2, labels)
print("Batch loss1 = ", l1.data, "\n Batch loss2 = ", l2.data)

Batch loss1 =  tensor(0.4966) 
 Batch loss2 =  tensor(1.2389)
